<div>
<div style="text-align:center; display:block; float:left; padding:80px;"><img width="200px"  src="https://kaggle2.blob.core.windows.net/competitions/kaggle/4651/logos/front_page.png"/><span style="color:red;">**New User Booking**</span></div>
<div style="">
**Objective:** In this recruiting competition, Airbnb challenges you to predict in which country a new user will make his or her first booking.  
  
** Description: ** In this challenge, you are given a list of users along with their demographics, web session records, and some summary statistics. You are asked to predict which country a new user's first booking destination will be. All the users in this dataset are from the USA.
</div>
<img src="https://kaggle2.blob.core.windows.net/competitions/kaggle/4651/media/airbnb_banner.png" />

Author: [Oliveira, D. M.](http://br.linkedin.com/in/dmoliveira)

## <span style="color:blue">Airbnb - New User Booking - v2 First Predictions</span>

## Import Packages

In [1]:
using DataFrames
using MLBase
using XGBoost

  likely near /Users/diego/.julia/v0.4/MLBase/src/modeltune.jl:5
  likely near /Users/diego/.julia/v0.4/MLBase/src/modeltune.jl:5
  likely near /Users/diego/.julia/v0.4/MLBase/src/modeltune.jl:5
  likely near /Users/diego/.julia/v0.4/MLBase/src/deprecated/datapre.jl:104
  likely near /Users/diego/.julia/v0.4/MLBase/src/deprecated/datapre.jl:105
  likely near /Users/diego/.julia/v0.4/MLBase/src/deprecated/datapre.jl:163
  likely near /Users/diego/.julia/v0.4/MLBase/src/deprecated/datapre.jl:163
  likely near /Users/diego/.julia/v0.4/MLBase/src/deprecated/datapre.jl:163


## Load Data

In [16]:
train = readtable("data/train_v2.tsv", separator='\t')
test  = readtable("data/test_v2.tsv", separator='\t')
full  = readtable("data/full_v1.tsv", separator='\t');

## Set Features and Output

In [17]:
label    = :country_destination
labels   = Set(train[label])
features = setdiff(names(test), [:id]);

original_labels = keys(labelmap(readtable("data/train_users_2.csv.gz")[label]));

## Prepare Instances

In [18]:
function split_train_val(df; train_size=.85, random_state=1)
    
    srand(random_state)
    
    nrows, ntraining_rows = size(df, 1), round(Int, size(df, 1) * train_size)
    indexes               = shuffle(collect(1:nrows))
    train                 = df[indexes[1:ntraining_rows], :]
    validation            = df[indexes[ntraining_rows+1:end], :]
    
    return train, validation
end

split_train_val (generic function with 1 method)

In [19]:
train[label]  -= 1
X_train, X_val = split_train_val(train, train_size=.85, random_state=1)

train_x = Array{Float64,2}(X_train[:, features])
train_y = Array{Float64,1}(X_train[label])
val_x   = Array{Float64,2}(X_val[:, features])
val_y   = Array{Float64,1}(X_val[label])
test_x  = Array{Float64,2}(test[:, features]);

In [20]:
dtrain  = DMatrix(train_x, label=train_y)
dval    = DMatrix(val_x, label=val_y);

## Train

In [ ]:
num_rounds = 150
watchlist  = [(dtrain, "train"), (dval, "eval")]
metrics    = ["merror", "mlogloss"]
params     = Dict("objective"         => "multi:softprob",
                   "booster"          => "gbtree",
                   "eta"              => .1,
                   "alpha"            => .5,
                   "gamma"            => .0,
                   "max_depth"        =>  5,
                   "colsample_bytree" => .5,
                   "min_child_weight" =>  10,
                   "subsample"        => .5,
                   "seed"             =>  1)

println("Training Base Model...")
tic()
model      = XGBoost.xgboost(dtrain, num_rounds, param=params, metrics=metrics,
                             num_class=length(labels), watchlist=watchlist)
toc()

println("\nTraining Boosted Model...")
ptrain = XGBoost.predict(model, dtrain, output_margin=true)
pval   = XGBoost.predict(model, dval, output_margin=true)
set_info(dtrain, "base_margin", ptrain)
set_info(dval, "base_margin", pval)

tic()
model = XGBoost.xgboost(dtrain, num_rounds, param=params, metrics=metrics,
                        num_class=length(labels), watchlist=watchlist)
toc()

Training Base Model...


[1]	train-merror:0.370963	train-mlogloss:1.199009	eval-merror:0.368636	eval-mlogloss:1.195946
[2]	train-merror:0.371074	train-mlogloss:1.187367	eval-merror:0.369167	eval-mlogloss:1.184286
[3]	train-merror:0.370804	train-mlogloss:1.176793	eval-merror:0.368511	eval-mlogloss:1.173735
[4]	train-merror:0.370677	train-mlogloss:1.167121	eval-merror:0.369105	eval-mlogloss:1.164166
[5]	train-merror:0.370594	train-mlogloss:1.158320	eval-merror:0.368324	eval-mlogloss:1.155501
[6]	train-merror:0.370467	train-mlogloss:1.150363	eval-merror:0.368824	eval-mlogloss:1.147608
[7]	train-merror:0.370396	train-mlogloss:1.143163	eval-merror:0.368262	eval-mlogloss:1.140408
[8]	train-merror:0.370247	train-mlogloss:1.136460	eval-merror:0.368449	eval-mlogloss:1.133799
[9]	train-merror:0.370109	train-mlogloss:1.130326	eval-merror:0.368324	eval-mlogloss:1.127812
[10]	train-merror:0.369641	train-mlogloss:1.124445	eval-merror:0.368355	eval-mlogloss:1.121969
[11]	train-merror:0.369348	train-mlogloss:1.118918	eval-mer

## Predict

In [42]:
function get_top_n(prob_matrix, n)
    
    top_n_list   = Array[]
    nrows, ncols = size(prob_matrix)
    n            = min(ncols, n)
    
    for i=1:nrows
        
        tuple_list = [(j, prob_matrix[i, j]) for j=1:ncols]
        top_n      = sort(tuple_list, by = x -> last(x), rev=true)[1:n]
        top_n      = [first(x) for x in top_n]
        
        push!(top_n_list, top_n)
    end
    
    return top_n_list
end

get_top_n (generic function with 1 method)

## Evaluate Train & Validation

In [134]:
function eval_map_precision_at_n(y, yhats; at_n=5)
    total, map_hit, precision_hit = 0, 0, 0
    by(yhats, :id) do df
        index = findfirst(df[:country], y[df[1, :id]]) - 1
        index >= 0 && (map_hit += (at_n - index)/at_n; precision_hit += 1)
        total += 1
    end
    return round(map_hit/total, 4), round(precision_hit/total, 4)
end

function evaluate_at_n(ids, labels, data; at_n=5)
    nrows = size(data, 1)
    y = Dict([ids[i] => original_labels[labels[i] + 1] for i=1:nrows])
    yhats = XGBoost.predict(model, data)
    yhats = reshape(yhats, length(original_labels), nrows)'
    yhats_top_n = prepare_dataframe_submission(ids, get_top_n(yhats, at_n))
    eval_map_precision_at_n(y, yhats_top_n)
end

function prepare_dataframe_submission(ids, yhats_top_n)
    
    ids_repeated  = repeach(ids, 5)
    new_yhats     = [ original_labels[yhat] for yhat in vcat(yhats_top_n...) ]
    submission_df = DataFrame(id=ids_repeated, country=new_yhats)
    
    return submission_df
end

prepare_dataframe_submission (generic function with 3 methods)

In [156]:
evaluate_at_n(X_train[:id], X_train[label], train_x, at_n=5)

(0.8642,0.9623)

In [157]:
evaluate_at_n(X_val[:id], X_val[label], val_x, at_n=5)

(0.8633,0.9599)

## Generate Submission File

In [75]:
yhats = XGBoost.predict(model, test_x)
yhats = reshape(yhats, length(original_labels), size(test_x, 1))';

In [76]:
yhats_top_n = get_top_n(yhats, 5);

In [77]:
submission_df = prepare_dataframe_submission(test[:id], yhats, yhats_top_n);

In [78]:
writetable("data/submissions/submission_v7_ds2_xgboost_msoftprob_gbtree_eta05_md5_ss3_alpha5_csb5_mcw10_nr150.csv", submission_df);

## Kagle Scores from Submited Predictions

Best Score: <span style="color:blue;">0.86389</span> [v7]

- v7 **XGB** (DS2 MSoftProb GBTree Eta.05 MD 6 SS.5 NR150 $\alpha$5 CSB5 MCW10): **0.86389** tme:.367	tmll:1.05 eme:.367	emll:1.06 (map/p@5: (0.8704,0.9667), (0.8685,0.9607))
- v6 **XGB** (DS2 MSoftProb GBTree Eta.3 MD 6 SS.5 NR100 $\alpha$5 CSB5 MCW10): **0.86359** tme.364 tmll1.04 eme.366 emll1.05   
- v4 **XGB** (DS1 MSoftProb GBTree Eta.3 MD 6 SS.5 NR100 $\alpha$5 CSB5 MCW10): **0.86109** tme.360 tmll1.02 eme.369 emll1.064   
- v3 **XGB** (DS1 MSoftMax GBTree Eta.3 MD 6 SS.5 NR100 $\alpha$5 CSB5 MCW10): **0.70496** tme 0.358 tmll 1.019 eme 0.366 emll1.061  
- v2 **XGB** (DS1 MSoftMax GBTree Eta.7 MD 5 SS.85 NR2000): ** 0.65697** tme.181 tmll.499 eme.403 emll1.477
- v1 **XGB** (DS1 MSoftMax GBTree Eta.7 MD 5 SS.85 NR100): **0.70174** tme.349 tmll.948 eme.367 emll1.077